In [1]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
from src.training.peft_bert_pipeline import BertPipelinePeft
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           4859 non-null   object 
 1   reference_answer   5363 non-null   object 
 2   answer             5363 non-null   object 
 3   score              5363 non-null   float64
 4   dataset            5363 non-null   object 
 5   normalized_score   5363 non-null   float64
 6   normalized_score2  5363 non-null   int64  
 7   bert_length        5363 non-null   int64  
 8   indobert_length    5363 non-null   int64  
 9   albert_length      5363 non-null   int64  
 10  longformer_length  5363 non-null   int64  
 11  multibert_length   5363 non-null   int64  
 12  indoalbert_length  5363 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 544.8+ KB


In [3]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

5
7.659122644421146e-05


In [4]:
results = []
results_epoch = []

In [5]:
batch_sizes = [4]
overlappings = [0, 256]
epochs_list = [5]
learning_rates = [1e-5, 5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [6]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-uncased")
# ]

In [ ]:
for batch_size in batch_sizes:
    for overlapping in overlappings:
        for num_epochs in epochs_list:
            for lr in learning_rates:
                df_result1 = None
                # Check if the second file exists
                if os.path.exists("experiments/results/results_epoch.csv"):
                    df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                    print(max(df_result1['valid_qwk']))
                else:
                    print("File 'results_epoch.csv' does not exist.")
                config = {
                    "df": df,
                    "model_name": "google-bert/bert-base-multilingual-uncased",
                    "overlapping": overlapping,
                    "batch_size": batch_size,
                    "learning_rate": lr,
                    "epochs": num_epochs,
                    "config_id": idx,
                    "max_seq_len": 512,
                    "col_length": "multibert_length",
                    "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf"),
                    "lora_rank": 16,
                    "lora_alpha": 16,
                }

                logging.info(
                    f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}, "
                    f"lora_rank:{config['lora_rank']}, lora_alpha:{config['lora_alpha']}"
                )
                
                print(
                    f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}, "
                    f"lora_rank:{config['lora_rank']}, lora_alpha:{config['lora_alpha']}"
                )
                
                try:
                    pipeline = BertPipelinePeft(config, results, results_epoch)
                    pipeline.run_training()

                    # Save results
                    # Dapatkan root project
                    results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                    results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                    TrainingBertPipeline.save_csv(results, results_path)
                    TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
                except Exception as e:
                    logging.error(f"Error in config_id={idx}: {str(e)}")
                    print(f"Error in config_id={idx}: {str(e)}")
                    torch.cuda.empty_cache()
                finally:
                    # Clear GPU memory after every configuration
                    del pipeline.model
                    del pipeline.tokenizer
                    del pipeline.optimizer
                    torch.cuda.empty_cache()

                idx += 1